# Changelog
- V2: 
 - Uploaded train/test images with Super Resolution which where produced on my own work-station. (See details on this ohase below)
 - Define Double size U-net to acommodate with the new image size: 202x202 (instead of 101x101)
 - Add blur image feature that categorieses image bluriness
 - Use same L/R data augmentation method as in baseline network. 
 
-V8:  Cancel second phase training, fix test phase bug

-V9: Fix file path of the test dataset

-V10: Fix bug in submission phase,

-V11: Yet another bug fix in submission phase,

-V12: Reduce augmented data to avoid memory overflow.

-V13 : One more bug fix

-V15 : Try out with less data

-V16 : Disabled all data mirroring

# About
Using the well written code of Peter, I wanted to see if using super resolution method that converts the training/testing images to x2 resolution can bring better results to the segmentation task and thus can be considered as a data augmentation method. The baseline comparison for improvement will be in relation to the original U-net (The Forked parent kernel). To accommodate with the higher resolution the new model network includes an additional stage so that it can handle double size image: 256x256 (instead of 128x128) 
In addition an image blurriness estimator is added in order to optimize the training process. It is assumed that the sharper images are then they would yield better super resolution images. The training data uses the same L/R data augmentation method as in baseline network. 

The super resolution data-set of images (SR images) were generated using the following network:
"Zero-Shot" Super-Resolution using Deep Internal Learning (ZSSR)'
Official implementation for paper by: Assaf Shocher, Nadav Cohen, Michal Irani
https://github.com/assafshocher/ZSSR

and was uploaded. to the local input folder. The code includes less graphic samples and illustrations (with you my apologies). 
See results in the Conclusions section


In [ ]:
import numpy as np
import pandas as pd
import cv2
from random import randint

import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
import seaborn as sns
sns.set_style("white")
from sklearn.model_selection import train_test_split

from skimage.transform import resize

from keras.preprocessing.image import load_img
from keras import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import load_model
from keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, Dropout

from tqdm import tqdm_notebook
from pathlib import Path
#from os import walk
#import zipfile

# Params and helpers

In [ ]:
img_size_ori = 101
img_size_super = 202
img_size_target = 128
Blur = 20

def upsampleSR(img):
    #if img_size_super == img_size_target*2:
    #    return img
    return resize(img, (img_size_target*2, img_size_target*2), mode='constant', preserve_range=True)

def downsample(img):
    if img_size_ori == img_size_target:
        return img
    return resize(img, (img_size_ori, img_size_ori), mode='constant', preserve_range=True)
    #return img[:img_size_ori, :img_size_ori]
    



# Loading of training/testing ids and depths
Reading the training data and the depths, store them in a DataFrame. Also create a test DataFrame with entries from depth not in train.

In [ ]:
train_df = pd.read_csv("../input/tgs-salt-identification-challenge/train.csv", index_col="id", usecols=[0])
depths_df = pd.read_csv("../input/tgs-salt-identification-challenge/depths.csv", index_col="id")
train_df = train_df.join(depths_df)
test_df = depths_df[~depths_df.index.isin(train_df.index)]
path_trainSuper = '../input/tsg-salt-superres-training-data/trainsuper/'
path_testSuper = '../input/tsg-salt-superres-training-data/testsuper/'

#print(check_output(["ls", "../input/tsg-salt-superres-training-data/testsuper/"]).decode("utf8"))

# Read images and masks
Load the images and masks into the DataFrame and divide the pixel values by 255.

In [ ]:
train_df["images"] = [np.array(load_img(path_trainSuper + "trainSuper/images/{}_zssr_X2.00X2.00.png".format(idx), grayscale=True)) / 255 for idx in tqdm_notebook(train_df.index)]


In [ ]:
train_df["masks"] = [np.array(load_img(path_trainSuper + "trainSuper/masks/{}_zssr_X2.00X2.00.png".format(idx), grayscale=True)) / 255 for idx in tqdm_notebook(train_df.index)]

# Calculating the salt coverage and salt coverage classes
Counting the number of salt pixels in the masks and dividing them by the image size. Also create 10 coverage classes, -0.1 having no salt at all to 1.0 being salt only.
Plotting the distribution of coverages and coverage classes, and the class against the raw coverage.

In [ ]:
train_df["coverage"] = train_df.masks.map(np.sum) / pow(img_size_super, 2)

In [ ]:
def cov_to_class(val):    
    for i in range(0, 10):
        if int(val * 10) <= i :
            return i
        
train_df["coverage_class"] = train_df.coverage.map(cov_to_class)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15,5))
sns.distplot(train_df.coverage, kde=False, ax=axs[0])
sns.distplot(train_df.coverage_class, bins=10, kde=False, ax=axs[1])
plt.suptitle("Salt coverage")
axs[0].set_xlabel("Coverage")
axs[1].set_xlabel("Coverage class")

In [ ]:
plt.scatter(train_df.coverage, train_df.coverage_class)
plt.xlabel("Coverage")
plt.ylabel("Coverage class")


# New Image Blur Classification
 An image bluriness estimator is added using a laplacian estimator (from open-cv library)

In [ ]:
def loadCalcImageBlur(img_path, tindex):
  f = []
  imBlurDict = {}
  for fl in tindex:
     if not Path(img_path+ fl+".png").exists():
       #print ( img_path+fl)
       continue
     img = cv2.imread(img_path+fl+".png", cv2.COLOR_BGR2GRAY)
     a =np.asarray(img)
     bl = np.max(cv2.convertScaleAbs(cv2.Laplacian(img,3)))
     #print (fl, "blur:", bl)
     imBlurDict[fl] = bl
  return imBlurDict

d_blur =loadCalcImageBlur('../input/tgs-salt-identification-challenge/train/images/',train_df.index)
train_df["id"]=train_df.index
#print ( train_df["id"].map(d_blur))
train_df["blur"]= train_df["id"].map(d_blur)
blur_df = train_df["blur"]

print ("Total train images:",len(train_df))
print ("Total test images:",len(test_df))




# Filter out blurry images

In [ ]:
if Blur >0:
  train_df = train_df[train_df["blur"] >= Blur]

# Create train/validation split stratified by salt coverage
Using the salt coverage as a stratification criterion. Also show an image to check for correct upsampling.

In [ ]:
ids_train, ids_valid, x_train, x_valid, y_train, y_valid, cov_train, cov_test, depth_train, depth_test = train_test_split(
    train_df.index.values,
    np.array(train_df.images.tolist()).reshape(-1, img_size_super, img_size_super, 1), 
    np.array(train_df.masks.tolist()).reshape(-1, img_size_super, img_size_super, 1), 
    train_df.coverage.values,
    train_df.z.values,
    test_size=0.2, stratify=train_df.coverage_class, random_state=1337)

In [ ]:
x_train_sr = np.array(train_df.images.loc[ids_train].map(upsampleSR).tolist()).reshape(-1, img_size_target*2, img_size_target*2, 1)
y_train_sr = np.array(train_df.masks.loc[ids_train].map(upsampleSR).tolist()).reshape(-1, img_size_target*2, img_size_target*2, 1)
print ("Shape upsample:",x_train_sr.shape) 

x_valid_sr = np.array(train_df.images.loc[ids_valid].map(upsampleSR).tolist()).reshape(-1, img_size_target*2, img_size_target*2, 1)
y_valid_sr = np.array(train_df.masks.loc[ids_valid].map(upsampleSR).tolist()).reshape(-1, img_size_target*2, img_size_target*2, 1)



# Build model

In [ ]:
def build_model(input_layer, start_neurons):
    # 256 -> 128
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(input_layer)
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)
    pool1 = Dropout(0.25)(pool1)
    # 128 -> 64
    conv1 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(input_layer)
    conv1 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)
    pool1 = Dropout(0.25)(pool1)

    # 64 -> 32
    conv2 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(pool1)
    conv2 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)
    pool2 = Dropout(0.5)(pool2)

    # 32 -> 16
    conv3 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(pool2)
    conv3 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(conv3)
    pool3 = MaxPooling2D((2, 2))(conv3)
    pool3 = Dropout(0.5)(pool3)

    # 16 -> 8
    conv4 = Conv2D(start_neurons * 16, (3, 3), activation="relu", padding="same")(pool3)
    conv4 = Conv2D(start_neurons * 16, (3, 3), activation="relu", padding="same")(conv4)
    pool4 = MaxPooling2D((2, 2))(conv4)
    pool4 = Dropout(0.5)(pool4)

    # Middle
    convm = Conv2D(start_neurons * 32, (3, 3), activation="relu", padding="same")(pool4)
    convm = Conv2D(start_neurons * 32, (3, 3), activation="relu", padding="same")(convm)

    # 8 -> 16
    deconv4 = Conv2DTranspose(start_neurons * 16, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv4 = concatenate([deconv4, conv4])
    uconv4 = Dropout(0.5)(uconv4)
    uconv4 = Conv2D(start_neurons * 16, (3, 3), activation="relu", padding="same")(uconv4)
    uconv4 = Conv2D(start_neurons * 16, (3, 3), activation="relu", padding="same")(uconv4)
    # 16-> 32
    deconv4 = Conv2DTranspose(start_neurons * 8, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv4 = concatenate([deconv4, conv4])
    uconv4 = Dropout(0.5)(uconv4)
    uconv4 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(uconv4)
    uconv4 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(uconv4)

    # 32 -> 64
    deconv3 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="same")(uconv4)
    uconv3 = concatenate([deconv3, conv3])
    uconv3 = Dropout(0.5)(uconv3)
    uconv3 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(uconv3)
    uconv3 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(uconv3)

    # 64 -> 128
    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(uconv3)
    uconv2 = concatenate([deconv2, conv2])
    uconv2 = Dropout(0.5)(uconv2)
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(uconv2)
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(uconv2)

    #128 -> 256    
    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    uconv1 = Dropout(0.5)(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(uconv1)

    #uconv1 = Dropout(0.5)(uconv1)
    output_layer = Conv2D(1, (1,1), padding="same", activation="sigmoid")(uconv1)
    
    return output_layer

input_layer = Input((img_size_target*2, img_size_target*2, 1))
output_layer = build_model(input_layer, 16)

In [ ]:
model = Model(input_layer, output_layer)

In [ ]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
model.summary()

# Data augmentation
Use data augmentation with only part of the mirrord images due to lack of memory

In [ ]:
'''
ltr = len(x_train_sr)
x_train_sr = np.append(x_train_sr, [np.fliplr(x) for x in x_train_sr[:ltr-2800]], axis=0)
y_train_sr = np.append(y_train_sr, [np.fliplr(x) for x in y_train_sr[:ltr-2800]], axis=0)
'''

In [ ]:
print(len(x_train_sr))

# Training

In [ ]:
early_stopping = EarlyStopping(patience=10, verbose=1)
model_checkpoint = ModelCheckpoint("./keras_SR.model", save_best_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(factor=0.14, patience=4, min_lr=0.000001, verbose=1)

epochs = 100
batch_size = 24 #32

history = model.fit(x_train_sr, y_train_sr,
                    validation_data=[x_valid_sr, y_valid_sr], 
                    epochs=epochs,
                    batch_size=batch_size,
                    callbacks=[early_stopping, model_checkpoint, reduce_lr])

## Second Phase Training (TBD) 
Second phase training with mirrored data due to lack of memory.

In [ ]:
'''
x_train_sr = np.array(list(map(np.fliplr, x_train_sr)))
y_train_sr = np.array(list(map(np.fliplr, y_train_sr)))
#y_train_sr = np.array([np.fliplr(x) for x in y_train_sr])
print(len(x_train_sr))
model = load_model("./keras_SR.model")
epochs = 30
batch_size = 24 #32
history = model.fit(x_train_sr, y_train_sr,
                    validation_data=[x_valid_sr, y_valid_sr], 
                    epochs=epochs,
                    batch_size=batch_size,
                    callbacks=[early_stopping, model_checkpoint, reduce_lr])
'''                    

In [ ]:
fig, (ax_loss, ax_acc) = plt.subplots(1, 2, figsize=(15,5))
ax_loss.plot(history.epoch, history.history["loss"], label="Train loss")
ax_loss.plot(history.epoch, history.history["val_loss"], label="Validation loss")
ax_acc.plot(history.epoch, history.history["acc"], label="Train accuracy")
ax_acc.plot(history.epoch, history.history["val_acc"], label="Validation accuracy")

In [ ]:
model = load_model("./keras_SR.model")

# Predict the validation set to do a sanity check
Again plot some sample images including the predictions.

In [ ]:
preds_valid_sr = model.predict(x_valid_sr).reshape(-1, img_size_target*2, img_size_target*2)
#preds_valid = np.array([downsample(x) for x in preds_valid])
#y_valid_ori = np.array([train_df.loc[idx].masks for idx in ids_valid])


# Scoring
Score the model and do a threshold optimization by the best IoU.

In [ ]:
# src: https://www.kaggle.com/aglotero/another-iou-metric
def iou_metric(y_true_in, y_pred_in, print_table=False):
    labels = y_true_in
    y_pred = y_pred_in
    
    true_objects = 2
    pred_objects = 2

    intersection = np.histogram2d(labels.flatten(), y_pred.flatten(), bins=(true_objects, pred_objects))[0]

    # Compute areas (needed for finding the union between all objects)
    area_true = np.histogram(labels, bins = true_objects)[0]
    area_pred = np.histogram(y_pred, bins = pred_objects)[0]
    area_true = np.expand_dims(area_true, -1)
    area_pred = np.expand_dims(area_pred, 0)

    # Compute union
    union = area_true + area_pred - intersection

    # Exclude background from the analysis
    intersection = intersection[1:,1:]
    union = union[1:,1:]
    union[union == 0] = 1e-9

    # Compute the intersection over union
    iou = intersection / union

    # Precision helper function
    def precision_at(threshold, iou):
        matches = iou > threshold
        true_positives = np.sum(matches, axis=1) == 1   # Correct objects
        false_positives = np.sum(matches, axis=0) == 0  # Missed objects
        false_negatives = np.sum(matches, axis=1) == 0  # Extra objects
        tp, fp, fn = np.sum(true_positives), np.sum(false_positives), np.sum(false_negatives)
        return tp, fp, fn

    # Loop over IoU thresholds
    prec = []
    if print_table:
        print("Thresh\tTP\tFP\tFN\tPrec.")
    for t in np.arange(0.5, 1.0, 0.05):
        tp, fp, fn = precision_at(t, iou)
        if (tp + fp + fn) > 0:
            p = tp / (tp + fp + fn)
        else:
            p = 0
        if print_table:
            print("{:1.3f}\t{}\t{}\t{}\t{:1.3f}".format(t, tp, fp, fn, p))
        prec.append(p)
    
    if print_table:
        print("AP\t-\t-\t-\t{:1.3f}".format(np.mean(prec)))
    return np.mean(prec)

def iou_metric_batch(y_true_in, y_pred_in):
    batch_size = y_true_in.shape[0]
    metric = []
    for batch in range(batch_size):
        value = iou_metric(y_true_in[batch], y_pred_in[batch])
        metric.append(value)
    return np.mean(metric)

In [ ]:
thresholds = np.linspace(0, 1, 50)
ious = np.array([iou_metric_batch(y_valid_sr, np.int32(preds_valid_sr > threshold)) for threshold in tqdm_notebook(thresholds)])

In [ ]:
threshold_best_index = np.argmax(ious[9:-10]) + 9
iou_best = ious[threshold_best_index]
threshold_best = thresholds[threshold_best_index]

In [ ]:
plt.plot(thresholds, ious)
plt.plot(threshold_best, iou_best, "xr", label="Best threshold")
plt.xlabel("Threshold")
plt.ylabel("IoU")
plt.title("Threshold vs IoU ({}, {})".format(threshold_best, iou_best))
plt.legend()

# Submission
Load, predict and submit the test image predictions.

In [ ]:
# Source https://www.kaggle.com/bguberfain/unet-with-depth
def RLenc(img, order='F', format=True):
    """
    img is binary mask image, shape (r,c)
    order is down-then-right, i.e. Fortran
    format determines if the order needs to be preformatted (according to submission rules) or not

    returns run length as an array or string (if format is True)
    """
    bytes = img.reshape(img.shape[0] * img.shape[1], order=order)
    runs = []  ## list of run lengths
    r = 0  ## the current run length
    pos = 1  ## count starts from 1 per WK
    for c in bytes:
        if (c == 0):
            if r != 0:
                runs.append((pos, r))
                pos += r
                r = 0
            pos += 1
        else:
            r += 1

    # if last run is unsaved (i.e. data ends with 1)
    if r != 0:
        runs.append((pos, r))
        pos += r
        r = 0

    if format:
        z = ''

        for rr in runs:
            z += '{} {} '.format(rr[0], rr[1])
        return z[:-1]
    else:
        return runs

## Dividing prediction task to batchs to reduce memory load
Clean up training data to save memory and start predictions on test images, doing so in batches.

In [ ]:


x_test = []
pred_idx = []
pred_dict = {}
lenl = len(test_df)
print (lenl)
for cnt, (idx, row) in enumerate(test_df.iterrows()):
  #print(cnt, idx, row.values)
 # print(cnt, test_df.loc[row])
  #arr_test = upsampleSR(np.array(load_img(zf_testSuper.open("testSuper/images/{}_zssr_X2.00X2.00.png".format(idx)), grayscale=True))) / 255 #.reshape(-1, img_size_target*2, img_size_target*2, 1)
  arr_test =  upsampleSR(np.array(load_img(path_testSuper + "testSuper/images/{}_zssr_X2.00X2.00.png".format(idx), grayscale=True))) / 255
  x_test.append(arr_test)
  pred_idx.append(idx)
  if (cnt % 3000 == 0)  :
    print(cnt, idx, row.values)
    #x_test_a = np.array(x_test).reshape(-1, img_size_target*2, img_size_target*2, 1)
    preds_test = model.predict(np.array(x_test).reshape(-1, img_size_target*2, img_size_target*2, 1))
    pred_dict.update({idx: RLenc(np.round(downsample(preds_test[i]) > threshold_best)) for i, idx in enumerate(tqdm_notebook(pred_idx))})
    pred_idx =[]
    x_test =[]
 
if (len(pred_idx)>0):
    preds_test = model.predict(np.array(x_test).reshape(-1, img_size_target*2, img_size_target*2, 1))
    pred_dict.update({idx: RLenc(np.round(downsample(preds_test[i]) > threshold_best)) for i, idx in enumerate(tqdm_notebook(pred_idx))})    

print (len(pred_dict))    
    

In [ ]:
sub = pd.DataFrame.from_dict(pred_dict,orient='index')
sub.index.names = ['id']
sub.columns = ['rle_mask']
sub.to_csv('submission.csv')

# Conclusions

A big problem that was encountered is the overloading of memory that is caused by the SR images and bigger NN. I could not run it through in the kaggle environment in a single batch. In an offline workstation that I used the training is performed in 2 phases - first one using the sharper images and the second shorter one using all images.
The results are not very conclusive but the SR method does attain about 1.5% precent improvment in the private score and 0.3% precent improvment in the public score.

These are the results :

Original net : Private: 0.737057 Public:0.710566

Big SR net :  Private 0.751683 Public: 0.713566




